In [ ]:
# Packages for data analysis
import pandas as pd
import numpy as np

In [ ]:
# Add Google Drive as an accessible path (Optional if you are running from Jupyter Notebook)
from google.colab import drive
drive.mount('/content/drive')

# change path to the designated google drive folder
# otherwise, data will be saved in /content folder which you may have issue locating
%cd /content/drive/My Drive/Colab Notebooks/CSS Project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/CSS Project


In [ ]:
# this will take some time
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 290 kB 5.3 MB/s 
     |████████████████████████████████| 2.9 MB 42.2 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 1.3 MB 46.9 MB/s 
     |████████████████████████████████| 125 kB 44.6 MB/s 
     |████████████████████████████████| 243 kB 34.6 MB/s 
     |████████████████████████████████| 160 kB 49.9 MB/s 
     |████████████████████████████████| 271 kB 42.0 MB/s 
     |████████████████████████████████| 3.3 MB 28.3 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 1.2 MB 36.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
  import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv('Tweets/gendersg_tweets.csv')
df.head()

,Unnamed: 0,date,content
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel wh..."
1,1,2020-10-22 14:36:12+00:00,exchange each other’s nudes w a friend from th...
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out t...
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the calif...
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal ...


In [ ]:
def zero_shot_classify_sequences(df, candidate_labels, threshold=0.5):
  #with classifier.device_placement():
    #outputs = classifier(examples['content'], candidate_labels, multi_label=True)

  batch_size = 200 #see how big you can make this number before out-of-memory
  aspect_classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0)
  sentence_list = list(df['content'])
  aspect_results = []
  for i in range(0, len(sentence_list), batch_size):
    #print(f'processing row {i} to {i+batch_size}')
    aspect_results += aspect_classifier(sentence_list[i:i+batch_size], candidate_labels,)

  final_outputs = []
  labels_set = set()
  for result in aspect_results:
    try:
      final_output = dict([(result['labels'][i], score) for i, score in enumerate(result['scores']) if score > threshold])
      labels_set.update(list(final_output.keys()))
    except TypeError:
      final_output = {}

    #print("final_output before return: \n", final_output)
    final_outputs.append(final_output)

  #print("Set of labels in threshold with this batch: ", len(labels_set), labels_set)
  df['predicted_labels'] = final_outputs

  return df

In [ ]:
candidate_labels = ['male','female']

In [ ]:
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel wh...",{'male': 0.6125432252883911}
1,1,2020-10-22 14:36:12+00:00,exchange each other’s nudes w a friend from th...,{'female': 0.5146311521530151}
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out t...,{'female': 0.5109924077987671}
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the calif...,{'male': 0.5736183524131775}
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal ...,{'female': 0.7635053992271423}
5,5,2020-09-09 02:11:58+00:00,What’s this gender reveal business?,{'female': 0.646432638168335}
6,6,2020-08-27 06:01:02+00:00,"Together, we can make gender equality a realit...",{'female': 0.9766485095024109}
7,7,2020-08-27 03:08:58+00:00,#JayaSaha \n#RheaChakrobarty \nYou both are a ...,{'female': 0.9665689468383789}
8,8,2020-07-30 11:47:56+00:00,Imiss being pregnant sia the milestone the mon...,{'female': 0.8797600865364075}
9,9,2020-06-29 17:00:24+00:00,My #GenderManifesto: I urge policy-makers and ...,{'female': 0.8648712038993835}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel wh...",{'male': 0.6125432252883911}
1,1,2020-10-22 14:36:12+00:00,exchange each other’s nudes w a friend from th...,{'female': 0.5146311521530151}
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out t...,{'female': 0.5109924077987671}
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the calif...,{'male': 0.5736183524131775}
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal ...,{'female': 0.7635053992271423}


In [ ]:
import os
os.makedirs('sentiment analysis', exist_ok=True)

In [ ]:
df.to_csv(f'sentiment analysis/gendersg_tweet_relevance.csv', index = False)

In [ ]:
df = pd.read_csv('sentiment analysis/gendersg_tweet_relevance.csv')
pd.set_option('display.max_colwidth', 150)
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel what we feel about them having girl bestfriends. I mean same goes with both genders but its UrGh",{'male': 0.6125432252883911}
1,1,2020-10-22 14:36:12+00:00,"exchange each other’s nudes w a friend from the opposite gender, is it something normal to do❓❓",{'female': 0.5146311521530151}
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out together and not be a couple okie thanks for coming to my ted talk bye stop being narrow minded love ...,{'female': 0.5109924077987671}
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the california fires has to be a fire breathing dragon,{'male': 0.5736183524131775}
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal parties like does it matter what gender your baby is why is it such a big deal you acting like if u ...,{'female': 0.7635053992271423}


In [ ]:
df.loc[df['predicted_labels'].str.contains('male|female')]

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel what we feel about them having girl bestfriends. I mean same goes with both genders but its UrGh",{'male': 0.6125432252883911}
1,1,2020-10-22 14:36:12+00:00,"exchange each other’s nudes w a friend from the opposite gender, is it something normal to do❓❓",{'female': 0.5146311521530151}
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out together and not be a couple okie thanks for coming to my ted talk bye stop being narrow minded love ...,{'female': 0.5109924077987671}
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the california fires has to be a fire breathing dragon,{'male': 0.5736183524131775}
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal parties like does it matter what gender your baby is why is it such a big deal you acting like if u ...,{'female': 0.7635053992271423}
5,5,2020-09-09 02:11:58+00:00,What’s this gender reveal business?,{'female': 0.646432638168335}
6,6,2020-08-27 06:01:02+00:00,"Together, we can make gender equality a reality. On Women's Equality Day, I am committing to taking action and helping create a more equal world f...",{'female': 0.9766485095024109}
7,7,2020-08-27 03:08:58+00:00,#JayaSaha \n#RheaChakrobarty \nYou both are a shame for my Bengali culture (which I have not forgotten even after living in Singapore for 25years ...,{'female': 0.9665689468383789}
8,8,2020-07-30 11:47:56+00:00,Imiss being pregnant sia the milestone the montly check up the gender reveal hahaahahha,{'female': 0.8797600865364075}
9,9,2020-06-29 17:00:24+00:00,My #GenderManifesto: I urge policy-makers and politicians in SG to close the gender pay gap.,{'female': 0.8648712038993835}


In [ ]:
candidate_labels = ['positive']
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel what we feel about them having girl bestfriends. I mean same goes with both genders but its UrGh",{}
1,1,2020-10-22 14:36:12+00:00,"exchange each other’s nudes w a friend from the opposite gender, is it something normal to do❓❓",{}
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out together and not be a couple okie thanks for coming to my ted talk bye stop being narrow minded love ...,{}
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the california fires has to be a fire breathing dragon,{}
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal parties like does it matter what gender your baby is why is it such a big deal you acting like if u ...,{}
5,5,2020-09-09 02:11:58+00:00,What’s this gender reveal business?,{}
6,6,2020-08-27 06:01:02+00:00,"Together, we can make gender equality a reality. On Women's Equality Day, I am committing to taking action and helping create a more equal world f...",{'positive': 0.9663419127464294}
7,7,2020-08-27 03:08:58+00:00,#JayaSaha \n#RheaChakrobarty \nYou both are a shame for my Bengali culture (which I have not forgotten even after living in Singapore for 25years ...,{}
8,8,2020-07-30 11:47:56+00:00,Imiss being pregnant sia the milestone the montly check up the gender reveal hahaahahha,{}
9,9,2020-06-29 17:00:24+00:00,My #GenderManifesto: I urge policy-makers and politicians in SG to close the gender pay gap.,{}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-17 12:11:05+00:00,"SOMETIMES, Men don’t understand why we feel what we feel about them having girl bestfriends. I mean same goes with both genders but its UrGh",{}
1,1,2020-10-22 14:36:12+00:00,"exchange each other’s nudes w a friend from the opposite gender, is it something normal to do❓❓",{}
2,2,2020-10-17 14:18:58+00:00,Two people of the opposite gender can go out together and not be a couple okie thanks for coming to my ted talk bye stop being narrow minded love ...,{}
3,3,2020-09-13 15:01:35+00:00,the gender of the child that started the california fires has to be a fire breathing dragon,{}
4,4,2020-09-12 16:13:09+00:00,i still dont get the concept of gender reveal parties like does it matter what gender your baby is why is it such a big deal you acting like if u ...,{}


In [ ]:
def relevance(rows, keyword):
  relevant = 0
  count = 0

  for i in rows:
    sentence = str(i)
    if keyword in i:
      relevant+=1
    count+=1

  return round((relevant/count)*100,2)

In [ ]:
relevance(df['predicted_labels'], 'positive')

16.67

"unfair"

In [ ]:
df = pd.read_csv('Tweets/unfairsg_tweets.csv')
df.head()

,Unnamed: 0,date,content
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...


In [ ]:
def zero_shot_classify_sequences(df, candidate_labels, threshold=0.5):
  #with classifier.device_placement():
    #outputs = classifier(examples['content'], candidate_labels, multi_label=True)

  batch_size = 200 #see how big you can make this number before out-of-memory
  aspect_classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0)
  sentence_list = list(df['content'])
  aspect_results = []
  for i in range(0, len(sentence_list), batch_size):
    #print(f'processing row {i} to {i+batch_size}')
    aspect_results += aspect_classifier(sentence_list[i:i+batch_size], candidate_labels,)

  final_outputs = []
  labels_set = set()
  for result in aspect_results:
    try:
      final_output = dict([(result['labels'][i], score) for i, score in enumerate(result['scores']) if score > threshold])
      labels_set.update(list(final_output.keys()))
    except TypeError:
      final_output = {}

    #print("final_output before return: \n", final_output)
    final_outputs.append(final_output)

  #print("Set of labels in threshold with this batch: ", len(labels_set), labels_set)
  df['predicted_labels'] = final_outputs

  return df

In [ ]:
candidate_labels = ['male','female']

In [ ]:
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}
5,5,2020-11-23 12:13:17+00:00,"When toxic person can no longer control u, they will try to control how others see u. The misinformation will feel unfair but stay above it. Trust...",{'male': 0.5040054321289062}
6,6,2020-10-22 11:39:12+00:00,Life is unfair.,{'male': 0.5185704231262207}
7,7,2020-10-02 06:52:13+00:00,why dont tutors just bump my 90% to a 100% this is so unfair U MIGHT AS WELL!!!!!!!! 😔🙄🙄🙄🙄,{'female': 0.6742623448371887}
8,8,2020-09-19 18:08:31+00:00,Alaaa you can say ref’s decision unfair to united or what la but you cannot deny united played like shit,{'male': 0.5713855028152466}
9,9,2020-09-19 18:04:12+00:00,Bro the foot on the line rule for goalkeepers is stupid and unfair,{'male': 0.8705627918243408}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}


In [ ]:
import os
os.makedirs('sentiment analysis', exist_ok=True)

In [ ]:
df.to_csv(f'sentiment analysis/gendersg_tweet_relevance.csv', index = False)

In [ ]:
df = pd.read_csv('sentiment analysis/gendersg_tweet_relevance.csv')
pd.set_option('display.max_colwidth', 150)
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}


In [ ]:
df.loc[df['predicted_labels'].str.contains('male|female')]

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}
5,5,2020-11-23 12:13:17+00:00,"When toxic person can no longer control u, they will try to control how others see u. The misinformation will feel unfair but stay above it. Trust...",{'male': 0.5040054321289062}
6,6,2020-10-22 11:39:12+00:00,Life is unfair.,{'male': 0.5185704231262207}
7,7,2020-10-02 06:52:13+00:00,why dont tutors just bump my 90% to a 100% this is so unfair U MIGHT AS WELL!!!!!!!! 😔🙄🙄🙄🙄,{'female': 0.6742623448371887}
8,8,2020-09-19 18:08:31+00:00,Alaaa you can say ref’s decision unfair to united or what la but you cannot deny united played like shit,{'male': 0.5713855028152466}
9,9,2020-09-19 18:04:12+00:00,Bro the foot on the line rule for goalkeepers is stupid and unfair,{'male': 0.8705627918243408}


In [ ]:
candidate_labels = ['positive']
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 05:25:55+00:00,"It’s always fun to eavesdrop on church youth groups having lunch discussions at the food court. Today’s discussion involved 4 youths (3F, 1M) disc...",{}
1,1,2020-12-30 04:41:10+00:00,stopppp ruininggggg women,{}
2,2,2020-12-30 03:47:38+00:00,"Without really planning on it, most of the books I've read this year have been by women authors, specifically BIPOC authors, and subsequently my l...",{'positive': 0.9128917455673218}
3,3,2020-12-26 04:31:20+00:00,I am starting to think there was a secret 14 men/women orgy at Mandarin Orchard,{}
4,4,2020-12-25 03:24:05+00:00,The Ripper docu clearly telling us — women that most men are bigots and misogynists 🤡,{}
...,...,...,...,...
271,271,2018-02-17 07:20:12+00:00,What the hell. Was loading my Korean show on Openload then there was this advertisement showing some 2D game that featured TOPLESS WOMEN CHARACTER...,{}
272,272,2018-02-14 03:16:57+00:00,While @chrissyteigen instagrams a pic of her salad with “NO DRESSING” and get 1.7million likes. We struggle to spread the idea of empowerment and ...,{'positive': 0.5007253885269165}
273,273,2018-02-11 16:08:45+00:00,Finding a good dinner and drinks place for 9 women without going OMG at the cost is Stress itself. Why am I even doing this??? Why why why??? #din...,{}
274,274,2018-02-11 12:37:43+00:00,"Geez, innocent tweet about an athlete and someone thought I said women as weak. Twitter is a hard place to live in.",{}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 05:25:55+00:00,"It’s always fun to eavesdrop on church youth groups having lunch discussions at the food court. Today’s discussion involved 4 youths (3F, 1M) disc...",{}
1,1,2020-12-30 04:41:10+00:00,stopppp ruininggggg women,{}
2,2,2020-12-30 03:47:38+00:00,"Without really planning on it, most of the books I've read this year have been by women authors, specifically BIPOC authors, and subsequently my l...",{'positive': 0.9128917455673218}
3,3,2020-12-26 04:31:20+00:00,I am starting to think there was a secret 14 men/women orgy at Mandarin Orchard,{}
4,4,2020-12-25 03:24:05+00:00,The Ripper docu clearly telling us — women that most men are bigots and misogynists 🤡,{}


In [ ]:
def relevance(rows, keyword):
  relevant = 0
  count = 0

  for i in rows:
    sentence = str(i)
    if keyword in i:
      relevant+=1
    count+=1

  return round((relevant/count)*100,2)

In [ ]:
relevance(df['predicted_labels'], 'positive')

23.55

"women"

In [ ]:
df = pd.read_csv('Tweets/womensg_tweets.csv')
df.head()

,Unnamed: 0,date,content
0,0,2020-12-30 05:25:55+00:00,"It’s always fun to eavesdrop on church youth groups having lunch discussions at the food court. Today’s discussion involved 4 youths (3F, 1M) disc..."
1,1,2020-12-30 04:41:10+00:00,stopppp ruininggggg women
2,2,2020-12-30 03:47:38+00:00,"Without really planning on it, most of the books I've read this year have been by women authors, specifically BIPOC authors, and subsequently my l..."
3,3,2020-12-26 04:31:20+00:00,I am starting to think there was a secret 14 men/women orgy at Mandarin Orchard
4,4,2020-12-25 03:24:05+00:00,The Ripper docu clearly telling us — women that most men are bigots and misogynists 🤡


In [ ]:
def zero_shot_classify_sequences(df, candidate_labels, threshold=0.5):
  #with classifier.device_placement():
    #outputs = classifier(examples['content'], candidate_labels, multi_label=True)

  batch_size = 200 #see how big you can make this number before out-of-memory
  aspect_classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0)
  sentence_list = list(df['content'])
  aspect_results = []
  for i in range(0, len(sentence_list), batch_size):
    #print(f'processing row {i} to {i+batch_size}')
    aspect_results += aspect_classifier(sentence_list[i:i+batch_size], candidate_labels,)

  final_outputs = []
  labels_set = set()
  for result in aspect_results:
    try:
      final_output = dict([(result['labels'][i], score) for i, score in enumerate(result['scores']) if score > threshold])
      labels_set.update(list(final_output.keys()))
    except TypeError:
      final_output = {}

    #print("final_output before return: \n", final_output)
    final_outputs.append(final_output)

  #print("Set of labels in threshold with this batch: ", len(labels_set), labels_set)
  df['predicted_labels'] = final_outputs

  return df

In [ ]:
candidate_labels = ['male','female']

In [ ]:
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}
5,5,2020-11-23 12:13:17+00:00,"When toxic person can no longer control u, they will try to control how others see u. The misinformation will feel unfair but stay above it. Trust...",{'male': 0.5040054321289062}
6,6,2020-10-22 11:39:12+00:00,Life is unfair.,{'male': 0.5185704231262207}
7,7,2020-10-02 06:52:13+00:00,why dont tutors just bump my 90% to a 100% this is so unfair U MIGHT AS WELL!!!!!!!! 😔🙄🙄🙄🙄,{'female': 0.6742623448371887}
8,8,2020-09-19 18:08:31+00:00,Alaaa you can say ref’s decision unfair to united or what la but you cannot deny united played like shit,{'male': 0.5713855028152466}
9,9,2020-09-19 18:04:12+00:00,Bro the foot on the line rule for goalkeepers is stupid and unfair,{'male': 0.8705627918243408}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}


In [ ]:
import os
os.makedirs('sentiment analysis', exist_ok=True)

In [ ]:
df.to_csv(f'sentiment analysis/gendersg_tweet_relevance.csv', index = False)

In [ ]:
df = pd.read_csv('sentiment analysis/gendersg_tweet_relevance.csv')
pd.set_option('display.max_colwidth', 150)
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}


In [ ]:
df.loc[df['predicted_labels'].str.contains('male|female')]

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-16 18:04:50+00:00,so fucking unfair fuck fuck fuck you,{'male': 0.5639718174934387}
1,1,2020-12-16 17:57:57+00:00,i fucking hate you. its always been you. never me. its so fucking unfair.,{'male': 0.6251437664031982}
2,2,2020-12-16 14:54:54+00:00,Can I just point out one thing?\n\nI just find it unfair and heartless that someone blames you for keeping quiet and not say a word when they were...,{'female': 0.5852227807044983}
3,3,2020-12-01 03:58:06+00:00,this is so unfair :D,{'female': 0.5513509511947632}
4,4,2020-11-29 09:45:07+00:00,Today someone asked if I'm bf-ing my baby and my heart breaks with sadness and embarrassment when I said no. Am I allowed to tell them the reason ...,{'female': 0.7347639799118042}
5,5,2020-11-23 12:13:17+00:00,"When toxic person can no longer control u, they will try to control how others see u. The misinformation will feel unfair but stay above it. Trust...",{'male': 0.5040054321289062}
6,6,2020-10-22 11:39:12+00:00,Life is unfair.,{'male': 0.5185704231262207}
7,7,2020-10-02 06:52:13+00:00,why dont tutors just bump my 90% to a 100% this is so unfair U MIGHT AS WELL!!!!!!!! 😔🙄🙄🙄🙄,{'female': 0.6742623448371887}
8,8,2020-09-19 18:08:31+00:00,Alaaa you can say ref’s decision unfair to united or what la but you cannot deny united played like shit,{'male': 0.5713855028152466}
9,9,2020-09-19 18:04:12+00:00,Bro the foot on the line rule for goalkeepers is stupid and unfair,{'male': 0.8705627918243408}


In [ ]:
candidate_labels = ['positive']
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 05:25:55+00:00,"It’s always fun to eavesdrop on church youth groups having lunch discussions at the food court. Today’s discussion involved 4 youths (3F, 1M) disc...",{}
1,1,2020-12-30 04:41:10+00:00,stopppp ruininggggg women,{}
2,2,2020-12-30 03:47:38+00:00,"Without really planning on it, most of the books I've read this year have been by women authors, specifically BIPOC authors, and subsequently my l...",{'positive': 0.9128917455673218}
3,3,2020-12-26 04:31:20+00:00,I am starting to think there was a secret 14 men/women orgy at Mandarin Orchard,{}
4,4,2020-12-25 03:24:05+00:00,The Ripper docu clearly telling us — women that most men are bigots and misogynists 🤡,{}
...,...,...,...,...
271,271,2018-02-17 07:20:12+00:00,What the hell. Was loading my Korean show on Openload then there was this advertisement showing some 2D game that featured TOPLESS WOMEN CHARACTER...,{}
272,272,2018-02-14 03:16:57+00:00,While @chrissyteigen instagrams a pic of her salad with “NO DRESSING” and get 1.7million likes. We struggle to spread the idea of empowerment and ...,{'positive': 0.5007253885269165}
273,273,2018-02-11 16:08:45+00:00,Finding a good dinner and drinks place for 9 women without going OMG at the cost is Stress itself. Why am I even doing this??? Why why why??? #din...,{}
274,274,2018-02-11 12:37:43+00:00,"Geez, innocent tweet about an athlete and someone thought I said women as weak. Twitter is a hard place to live in.",{}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 05:25:55+00:00,"It’s always fun to eavesdrop on church youth groups having lunch discussions at the food court. Today’s discussion involved 4 youths (3F, 1M) disc...",{}
1,1,2020-12-30 04:41:10+00:00,stopppp ruininggggg women,{}
2,2,2020-12-30 03:47:38+00:00,"Without really planning on it, most of the books I've read this year have been by women authors, specifically BIPOC authors, and subsequently my l...",{'positive': 0.9128917455673218}
3,3,2020-12-26 04:31:20+00:00,I am starting to think there was a secret 14 men/women orgy at Mandarin Orchard,{}
4,4,2020-12-25 03:24:05+00:00,The Ripper docu clearly telling us — women that most men are bigots and misogynists 🤡,{}


In [ ]:
def relevance(rows, keyword):
  relevant = 0
  count = 0

  for i in rows:
    sentence = str(i)
    if keyword in i:
      relevant+=1
    count+=1

  return round((relevant/count)*100,2)

In [ ]:
relevance(df['predicted_labels'], 'positive')

23.55

"equal"

In [ ]:
df = pd.read_csv('Tweets/equalsg_tweets.csv')
df.head()

,Unnamed: 0,date,content
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking..."
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race."
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...


In [ ]:
def zero_shot_classify_sequences(df, candidate_labels, threshold=0.5):
  #with classifier.device_placement():
    #outputs = classifier(examples['content'], candidate_labels, multi_label=True)

  batch_size = 200 #see how big you can make this number before out-of-memory
  aspect_classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0)
  sentence_list = list(df['content'])
  aspect_results = []
  for i in range(0, len(sentence_list), batch_size):
    #print(f'processing row {i} to {i+batch_size}')
    aspect_results += aspect_classifier(sentence_list[i:i+batch_size], candidate_labels,)

  final_outputs = []
  labels_set = set()
  for result in aspect_results:
    try:
      final_output = dict([(result['labels'][i], score) for i, score in enumerate(result['scores']) if score > threshold])
      labels_set.update(list(final_output.keys()))
    except TypeError:
      final_output = {}

    #print("final_output before return: \n", final_output)
    final_outputs.append(final_output)

  #print("Set of labels in threshold with this batch: ", len(labels_set), labels_set)
  df['predicted_labels'] = final_outputs

  return df

In [ ]:
candidate_labels = ['male','female']

In [ ]:
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...,{'male': 0.6342694163322449}
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂,{'female': 0.6000853180885315}
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking...",{'male': 0.6878974437713623}
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race.",{'male': 0.649141788482666}
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...,{'male': 0.6095405220985413}
5,5,2020-04-14 15:07:01+00:00,i’m telling y’all - age does not equal maturity okay 😂,{'female': 0.6052848100662231}
6,6,2020-04-04 13:06:50+00:00,"""People are not clones in a monoculture, so what satisfies one will frustrate another, and the only way they can end up equal is if they are treat...",{'male': 0.5258857607841492}
7,7,2020-03-17 00:20:28+00:00,How lovely to use my phone freely on the train. This is the best me time without anyone but myself l. Kids head home today equal less laundry duti...,{'female': 0.7986504435539246}
8,8,2020-03-08 11:43:08+00:00,Happy International Women’s Day to all of you strong women out there. Enjoy it have a good one. May God be with you. #Equal,{'female': 0.99029141664505}
9,9,2020-01-05 08:36:57+00:00,"Some of y’all rlly think rights are a piece of fking pie like as if the marginalised group of ppl get equal rights, it feels like you’re getting l...",{'male': 0.6820383071899414}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...,{'male': 0.6342694163322449}
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂,{'female': 0.6000853180885315}
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking...",{'male': 0.6878974437713623}
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race.",{'male': 0.649141788482666}
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...,{'male': 0.6095405220985413}


In [ ]:
import os
os.makedirs('sentiment analysis', exist_ok=True)

In [ ]:
df.to_csv(f'sentiment analysis/gendersg_tweet_relevance.csv', index = False)

In [ ]:
df = pd.read_csv('sentiment analysis/gendersg_tweet_relevance.csv')
pd.set_option('display.max_colwidth', 150)
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...,{'male': 0.6342694163322449}
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂,{'female': 0.6000853180885315}
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking...",{'male': 0.6878974437713623}
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race.",{'male': 0.649141788482666}
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...,{'male': 0.6095405220985413}


In [ ]:
df.loc[df['predicted_labels'].str.contains('male|female')]

,Unnamed: 0,date,content,predicted_labels
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...,{'male': 0.6342694163322449}
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂,{'female': 0.6000853180885315}
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking...",{'male': 0.6878974437713623}
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race.",{'male': 0.649141788482666}
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...,{'male': 0.6095405220985413}
5,5,2020-04-14 15:07:01+00:00,i’m telling y’all - age does not equal maturity okay 😂,{'female': 0.6052848100662231}
6,6,2020-04-04 13:06:50+00:00,"""People are not clones in a monoculture, so what satisfies one will frustrate another, and the only way they can end up equal is if they are treat...",{'male': 0.5258857607841492}
7,7,2020-03-17 00:20:28+00:00,How lovely to use my phone freely on the train. This is the best me time without anyone but myself l. Kids head home today equal less laundry duti...,{'female': 0.7986504435539246}
8,8,2020-03-08 11:43:08+00:00,Happy International Women’s Day to all of you strong women out there. Enjoy it have a good one. May God be with you. #Equal,{'female': 0.99029141664505}
9,9,2020-01-05 08:36:57+00:00,"Some of y’all rlly think rights are a piece of fking pie like as if the marginalised group of ppl get equal rights, it feels like you’re getting l...",{'male': 0.6820383071899414}


In [ ]:
candidate_labels = ['positive']
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...,{}
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂,{}
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking...",{}
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race.",{}
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...,{}
5,5,2020-04-14 15:07:01+00:00,i’m telling y’all - age does not equal maturity okay 😂,{}
6,6,2020-04-04 13:06:50+00:00,"""People are not clones in a monoculture, so what satisfies one will frustrate another, and the only way they can end up equal is if they are treat...",{}
7,7,2020-03-17 00:20:28+00:00,How lovely to use my phone freely on the train. This is the best me time without anyone but myself l. Kids head home today equal less laundry duti...,{'positive': 0.6232397556304932}
8,8,2020-03-08 11:43:08+00:00,Happy International Women’s Day to all of you strong women out there. Enjoy it have a good one. May God be with you. #Equal,{'positive': 0.9417197108268738}
9,9,2020-01-05 08:36:57+00:00,"Some of y’all rlly think rights are a piece of fking pie like as if the marginalised group of ppl get equal rights, it feels like you’re getting l...",{}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-09-28 13:07:35+00:00,even if u wna say religion isnt equal to human rights... a basic human right is having beliefs that deserve t b respected too? so u disrespect me ...,{}
1,1,2020-06-11 07:48:57+00:00,4 hours of Bollywood is like equal to 16episodes of Kdrama 😂,{}
2,2,2020-06-10 01:22:46+00:00,"You say black lives matter, but segregate the with handouts. I think blacks whites or blues, just want equal opportunities when they come knocking...",{}
3,3,2020-06-02 13:05:23+00:00,"I just hope that we will learn our lesson and acknowledge that we are all equal. As children of the earth, theres no such thing as a superior race.",{}
4,4,2020-04-16 12:18:53+00:00,ZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY\nZOOM CALLS DON'T EQUAL PRODUCTIVITY...,{}


In [ ]:
def relevance(rows, keyword):
  relevant = 0
  count = 0

  for i in rows:
    sentence = str(i)
    if keyword in i:
      relevant+=1
    count+=1

  return round((relevant/count)*100,2)

In [ ]:
relevance(df['predicted_labels'], 'positive')

14.29

"men"

In [ ]:
df = pd.read_csv('Tweets/mensg_tweets.csv')
df.head()

,Unnamed: 0,date,content
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit"
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?


In [ ]:
def zero_shot_classify_sequences(df, candidate_labels, threshold=0.5):
  #with classifier.device_placement():
    #outputs = classifier(examples['content'], candidate_labels, multi_label=True)

  batch_size = 200 #see how big you can make this number before out-of-memory
  aspect_classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3", device=0)
  sentence_list = list(df['content'])
  aspect_results = []
  for i in range(0, len(sentence_list), batch_size):
    #print(f'processing row {i} to {i+batch_size}')
    aspect_results += aspect_classifier(sentence_list[i:i+batch_size], candidate_labels,)

  final_outputs = []
  labels_set = set()
  for result in aspect_results:
    try:
      final_output = dict([(result['labels'][i], score) for i, score in enumerate(result['scores']) if score > threshold])
      labels_set.update(list(final_output.keys()))
    except TypeError:
      final_output = {}

    #print("final_output before return: \n", final_output)
    final_outputs.append(final_output)

  #print("Set of labels in threshold with this batch: ", len(labels_set), labels_set)
  df['predicted_labels'] = final_outputs

  return df

In [ ]:
candidate_labels = ['male','female']

In [ ]:
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit",{'male': 0.7715070247650146}
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke,{'male': 0.985879123210907}
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men,{'male': 0.9499832391738892}
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?,{'male': 0.9912464618682861}
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?,{'male': 0.9826486706733704}
...,...,...,...,...
484,484,2018-02-13 00:55:19+00:00,i think i was one of the last men standing but i just opened snapchat for the second time this morning and it updated. i’m offended.,{'male': 0.9919379353523254}
485,485,2018-02-12 12:46:22+00:00,Saw a clip of oily Turkish men in leather trousers wrestling ....yum,{'male': 0.9932325482368469}
486,486,2018-02-12 09:32:26+00:00,Men’s here always use shorts and it’s kinda too short btw . Sexy 🤤,{'male': 0.9929814338684082}
487,487,2018-02-11 17:56:58+00:00,I can't stop thinking about my boy\n\nSee this is the reason I have sex with men who are not my type\n\nIf they are my type I will linger over the...,{'male': 0.9259257912635803}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit",{'male': 0.7715070247650146}
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke,{'male': 0.985879123210907}
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men,{'male': 0.9499832391738892}
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?,{'male': 0.9912464618682861}
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?,{'male': 0.9826486706733704}


In [ ]:
import os
os.makedirs('sentiment analysis', exist_ok=True)

In [ ]:
df.to_csv(f'sentiment analysis/gendersg_tweet_relevance.csv', index = False)

In [ ]:
df = pd.read_csv('sentiment analysis/gendersg_tweet_relevance.csv')
pd.set_option('display.max_colwidth', 150)
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit",{'male': 0.7715070247650146}
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke,{'male': 0.985879123210907}
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men,{'male': 0.9499832391738892}
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?,{'male': 0.9912464618682861}
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?,{'male': 0.9826486706733704}


In [ ]:
df.loc[df['predicted_labels'].str.contains('male|female')]

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit",{'male': 0.7715070247650146}
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke,{'male': 0.985879123210907}
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men,{'male': 0.9499832391738892}
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?,{'male': 0.9912464618682861}
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?,{'male': 0.9826486706733704}
...,...,...,...,...
484,484,2018-02-13 00:55:19+00:00,i think i was one of the last men standing but i just opened snapchat for the second time this morning and it updated. i’m offended.,{'male': 0.9919379353523254}
485,485,2018-02-12 12:46:22+00:00,Saw a clip of oily Turkish men in leather trousers wrestling ....yum,{'male': 0.9932325482368469}
486,486,2018-02-12 09:32:26+00:00,Men’s here always use shorts and it’s kinda too short btw . Sexy 🤤,{'male': 0.9929814338684082}
487,487,2018-02-11 17:56:58+00:00,I can't stop thinking about my boy\n\nSee this is the reason I have sex with men who are not my type\n\nIf they are my type I will linger over the...,{'male': 0.9259257912635803}


In [ ]:
candidate_labels = ['positive']
zero_shot_classify_sequences(df, candidate_labels, threshold=0.5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit",{}
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke,{}
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men,{}
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?,{}
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?,{}
...,...,...,...,...
484,484,2018-02-13 00:55:19+00:00,i think i was one of the last men standing but i just opened snapchat for the second time this morning and it updated. i’m offended.,{}
485,485,2018-02-12 12:46:22+00:00,Saw a clip of oily Turkish men in leather trousers wrestling ....yum,{'positive': 0.609572172164917}
486,486,2018-02-12 09:32:26+00:00,Men’s here always use shorts and it’s kinda too short btw . Sexy 🤤,{}
487,487,2018-02-11 17:56:58+00:00,I can't stop thinking about my boy\n\nSee this is the reason I have sex with men who are not my type\n\nIf they are my type I will linger over the...,{}


In [ ]:
df.head()

,Unnamed: 0,date,content,predicted_labels
0,0,2020-12-30 04:40:43+00:00,"i would spend my last $5 on men but they wouldnt do the same , piece of shit",{}
1,1,2020-12-30 04:38:12+00:00,ive wasted alot of money trying to support men bc they are fkin broke,{}
2,2,2020-12-30 04:37:35+00:00,i need to stop spending my money on men,{}
3,3,2020-12-30 01:56:02+00:00,Just came to my mind: Why is it ok for @TheEllenShow to objectify men when it isn’t for anyone else? Double standards?,{}
4,4,2020-12-26 15:28:04+00:00,Does anyone knows where I can get mens jumpsuit?,{}


In [ ]:
def relevance(rows, keyword):
  relevant = 0
  count = 0

  for i in rows:
    sentence = str(i)
    if keyword in i:
      relevant+=1
    count+=1

  return round((relevant/count)*100,2)

In [ ]:
relevance(df['predicted_labels'], 'positive')

22.49

Adapted from: https://discuss.huggingface.co/t/apply-batched-zero-shot-classification-on-huggingface-datasets-object/5211/5